In [ ]:
## Train with validation
work_dir = 'work_dirs/00_20191018'

In [1]:
from __future__ import division
import argparse
import os

import torch
from mmcv import Config

from mmdet import __version__
from mmdet.apis import (get_root_logger, init_dist, set_random_seed,
                        train_detector)
from mmdet.datasets import build_dataset
from mmdet.models import build_detector

In [2]:
config_file = 'configs/00_faster_rcnn_r50_fpn_1x-SAU.py'
checkpoint_file = 'checkpoints/faster_rcnn_r50_fpn_1x_20181010-3d1b3351.pth'
gpus = 2

In [3]:
def train(config, gpus, autoscale_lr=False, launcher='none', local_rank=0, resume_from=None, seed=None, validate=False, work_dir=None):
    
    
    cfg = Config.fromfile(config_file)
    # set cudnn_benchmark
    if cfg.get('cudnn_benchmark', False):
        torch.backends.cudnn.benchmark = True
    # update configs according to CLI args
    if work_dir is not None:
        cfg.work_dir = work_dir
    if resume_from is not None:
        cfg.resume_from = resume_from
    cfg.gpus = gpus

    if autoscale_lr:
        # apply the linear scaling rule (https://arxiv.org/abs/1706.02677)
        cfg.optimizer['lr'] = cfg.optimizer['lr'] * cfg.gpus / 8

    # init distributed env first, since logger depends on the dist info.
    if launcher == 'none':
        distributed = False
    else:
        distributed = True
        init_dist(launcher, **cfg.dist_params)

    # init logger before other steps
    logger = get_root_logger(cfg.log_level)
    logger.info('Distributed training: {}'.format(distributed))

    # set random seeds
    if seed is not None:
        logger.info('Set random seed to {}'.format(seed))
        set_random_seed(seed)

    model = build_detector(
        cfg.model, train_cfg=cfg.train_cfg, test_cfg=cfg.test_cfg)

    datasets = [build_dataset(cfg.data.train)]
    if len(cfg.workflow) == 2:
        datasets.append(build_dataset(cfg.data.val))
    if cfg.checkpoint_config is not None:
        # save mmdet version, config file content and class names in
        # checkpoints as meta data
        cfg.checkpoint_config.meta = dict(
            mmdet_version=__version__,
            config=cfg.text,
            CLASSES=datasets[0].CLASSES)
    # add an attribute for visualization convenience
    model.CLASSES = datasets[0].CLASSES
    train_detector(
        model,
        datasets,
        cfg,
        distributed=distributed,
        validate=validate,
        logger=logger)

In [4]:
train(config_file, gpus, validate=True, work_dir=work_dir)

2019-10-17 22:32:25,590 - INFO - Distributed training: False
2019-10-17 22:32:25,794 - INFO - load model from: torchvision://resnet50
2019-10-17 22:32:25,916 - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias



loading annotations into memory...
Done (t=0.09s)
creating index...
index created!


2019-10-17 22:32:28,183 - INFO - Start running, host: thaikari@gislyf, work_dir: /home/thaikari/mmdetection/work_dirs/faster_rcnn_r50_fpn_1x
2019-10-17 22:32:28,183 - INFO - workflow: [('train', 1)], max: 12 epochs
2019-10-17 22:32:52,598 - INFO - Epoch [1][50/4269]	lr: 0.00299, eta: 6:56:23, time: 0.488, data_time: 0.012, memory: 2520, loss_rpn_cls: 0.2006, loss_rpn_bbox: 0.0125, loss_cls: 0.1755, acc: 95.9512, loss_bbox: 0.0637, loss: 0.4522
2019-10-17 22:33:14,320 - INFO - Epoch [1][100/4269]	lr: 0.00349, eta: 6:33:05, time: 0.434, data_time: 0.004, memory: 2520, loss_rpn_cls: 0.0394, loss_rpn_bbox: 0.0098, loss_cls: 0.1286, acc: 95.9756, loss_bbox: 0.1002, loss: 0.2780
2019-10-17 22:33:36,349 - INFO - Epoch [1][150/4269]	lr: 0.00399, eta: 6:26:49, time: 0.441, data_time: 0.004, memory: 2520, loss_rpn_cls: 0.0400, loss_rpn_bbox: 0.0107, loss_cls: 0.1090, acc: 96.1543, loss_bbox: 0.0964, loss: 0.2560
2019-10-17 22:33:58,140 - INFO - Epoch [1][200/4269]	lr: 0.00449, eta: 6:22:29, time

2019-10-17 22:45:40,304 - INFO - Epoch [1][1800/4269]	lr: 0.00750, eta: 6:02:31, time: 0.437, data_time: 0.005, memory: 2520, loss_rpn_cls: 0.0051, loss_rpn_bbox: 0.0049, loss_cls: 0.0449, acc: 98.2432, loss_bbox: 0.0455, loss: 0.1003
2019-10-17 22:46:02,138 - INFO - Epoch [1][1850/4269]	lr: 0.00750, eta: 6:02:04, time: 0.437, data_time: 0.005, memory: 2520, loss_rpn_cls: 0.0065, loss_rpn_bbox: 0.0061, loss_cls: 0.0488, acc: 98.1455, loss_bbox: 0.0477, loss: 0.1092
2019-10-17 22:46:23,966 - INFO - Epoch [1][1900/4269]	lr: 0.00750, eta: 6:01:38, time: 0.437, data_time: 0.005, memory: 2520, loss_rpn_cls: 0.0058, loss_rpn_bbox: 0.0047, loss_cls: 0.0478, acc: 98.1768, loss_bbox: 0.0495, loss: 0.1078
2019-10-17 22:46:45,762 - INFO - Epoch [1][1950/4269]	lr: 0.00750, eta: 6:01:11, time: 0.436, data_time: 0.005, memory: 2520, loss_rpn_cls: 0.0055, loss_rpn_bbox: 0.0060, loss_cls: 0.0486, acc: 98.2041, loss_bbox: 0.0499, loss: 0.1101
2019-10-17 22:47:07,545 - INFO - Epoch [1][2000/4269]	lr: 0.

2019-10-17 22:58:20,052 - INFO - Epoch [1][3550/4269]	lr: 0.00750, eta: 5:47:21, time: 0.433, data_time: 0.005, memory: 2520, loss_rpn_cls: 0.0036, loss_rpn_bbox: 0.0044, loss_cls: 0.0349, acc: 98.6084, loss_bbox: 0.0392, loss: 0.0821
2019-10-17 22:58:41,665 - INFO - Epoch [1][3600/4269]	lr: 0.00750, eta: 5:46:56, time: 0.432, data_time: 0.005, memory: 2520, loss_rpn_cls: 0.0048, loss_rpn_bbox: 0.0035, loss_cls: 0.0359, acc: 98.6436, loss_bbox: 0.0371, loss: 0.0813
2019-10-17 22:59:03,344 - INFO - Epoch [1][3650/4269]	lr: 0.00750, eta: 5:46:32, time: 0.434, data_time: 0.005, memory: 2520, loss_rpn_cls: 0.0027, loss_rpn_bbox: 0.0040, loss_cls: 0.0356, acc: 98.6260, loss_bbox: 0.0375, loss: 0.0798
2019-10-17 22:59:25,005 - INFO - Epoch [1][3700/4269]	lr: 0.00750, eta: 5:46:08, time: 0.433, data_time: 0.005, memory: 2520, loss_rpn_cls: 0.0046, loss_rpn_bbox: 0.0044, loss_cls: 0.0404, acc: 98.4395, loss_bbox: 0.0426, loss: 0.0920
2019-10-17 22:59:46,671 - INFO - Epoch [1][3750/4269]	lr: 0.

...

2019-10-18 03:50:00,078 - INFO - Epoch [11][1300/4269]	lr: 0.00075, eta: 0:52:00, time: 0.433, data_time: 0.005, memory: 2520, loss_rpn_cls: 0.0002, loss_rpn_bbox: 0.0009, loss_cls: 0.0107, acc: 99.6016, loss_bbox: 0.0090, loss: 0.0209
2019-10-18 03:50:21,680 - INFO - Epoch [11][1350/4269]	lr: 0.00075, eta: 0:51:39, time: 0.432, data_time: 0.004, memory: 2520, loss_rpn_cls: 0.0002, loss_rpn_bbox: 0.0009, loss_cls: 0.0098, acc: 99.6299, loss_bbox: 0.0085, loss: 0.0195
2019-10-18 03:50:43,296 - INFO - Epoch [11][1400/4269]	lr: 0.00075, eta: 0:51:17, time: 0.432, data_time: 0.004, memory: 2520, loss_rpn_cls: 0.0003, loss_rpn_bbox: 0.0010, loss_cls: 0.0089, acc: 99.6445, loss_bbox: 0.0082, loss: 0.0183
2019-10-18 03:51:04,896 - INFO - Epoch [11][1450/4269]	lr: 0.00075, eta: 0:50:56, time: 0.432, data_time: 0.004, memory: 2520, loss_rpn_cls: 0.0002, loss_rpn_bbox: 0.0008, loss_cls: 0.0098, acc: 99.6133, loss_bbox: 0.0083, loss: 0.0191
2019-10-18 03:51:26,561 - INFO - Epoch [11][1500/4269]	l

2019-10-18 04:02:36,650 - INFO - Epoch [11][3050/4269]	lr: 0.00075, eta: 0:39:26, time: 0.433, data_time: 0.004, memory: 2520, loss_rpn_cls: 0.0004, loss_rpn_bbox: 0.0009, loss_cls: 0.0096, acc: 99.6318, loss_bbox: 0.0079, loss: 0.0188
2019-10-18 04:02:58,236 - INFO - Epoch [11][3100/4269]	lr: 0.00075, eta: 0:39:04, time: 0.432, data_time: 0.004, memory: 2520, loss_rpn_cls: 0.0002, loss_rpn_bbox: 0.0009, loss_cls: 0.0099, acc: 99.6416, loss_bbox: 0.0082, loss: 0.0192
2019-10-18 04:03:19,829 - INFO - Epoch [11][3150/4269]	lr: 0.00075, eta: 0:38:43, time: 0.432, data_time: 0.004, memory: 2520, loss_rpn_cls: 0.0002, loss_rpn_bbox: 0.0010, loss_cls: 0.0096, acc: 99.6279, loss_bbox: 0.0092, loss: 0.0200
2019-10-18 04:03:41,401 - INFO - Epoch [11][3200/4269]	lr: 0.00075, eta: 0:38:21, time: 0.431, data_time: 0.004, memory: 2520, loss_rpn_cls: 0.0003, loss_rpn_bbox: 0.0009, loss_cls: 0.0097, acc: 99.6182, loss_bbox: 0.0086, loss: 0.0194
2019-10-18 04:04:03,008 - INFO - Epoch [11][3250/4269]	l

2019-10-18 04:15:22,238 - INFO - Epoch [12][550/4269]	lr: 0.00008, eta: 0:26:43, time: 0.433, data_time: 0.005, memory: 2520, loss_rpn_cls: 0.0003, loss_rpn_bbox: 0.0007, loss_cls: 0.0093, acc: 99.6426, loss_bbox: 0.0076, loss: 0.0179
2019-10-18 04:15:43,865 - INFO - Epoch [12][600/4269]	lr: 0.00008, eta: 0:26:21, time: 0.433, data_time: 0.005, memory: 2520, loss_rpn_cls: 0.0002, loss_rpn_bbox: 0.0010, loss_cls: 0.0102, acc: 99.6348, loss_bbox: 0.0095, loss: 0.0209
2019-10-18 04:16:05,558 - INFO - Epoch [12][650/4269]	lr: 0.00008, eta: 0:26:00, time: 0.434, data_time: 0.005, memory: 2520, loss_rpn_cls: 0.0002, loss_rpn_bbox: 0.0009, loss_cls: 0.0097, acc: 99.6221, loss_bbox: 0.0076, loss: 0.0185
2019-10-18 04:16:27,176 - INFO - Epoch [12][700/4269]	lr: 0.00008, eta: 0:25:38, time: 0.432, data_time: 0.005, memory: 2520, loss_rpn_cls: 0.0004, loss_rpn_bbox: 0.0010, loss_cls: 0.0084, acc: 99.6631, loss_bbox: 0.0077, loss: 0.0174
2019-10-18 04:16:48,797 - INFO - Epoch [12][750/4269]	lr: 0.

2019-10-18 04:27:59,406 - INFO - Epoch [12][2300/4269]	lr: 0.00008, eta: 0:14:08, time: 0.433, data_time: 0.005, memory: 2520, loss_rpn_cls: 0.0002, loss_rpn_bbox: 0.0008, loss_cls: 0.0084, acc: 99.6768, loss_bbox: 0.0077, loss: 0.0171
2019-10-18 04:28:21,011 - INFO - Epoch [12][2350/4269]	lr: 0.00008, eta: 0:13:47, time: 0.432, data_time: 0.005, memory: 2520, loss_rpn_cls: 0.0004, loss_rpn_bbox: 0.0009, loss_cls: 0.0100, acc: 99.6035, loss_bbox: 0.0090, loss: 0.0202
2019-10-18 04:28:42,700 - INFO - Epoch [12][2400/4269]	lr: 0.00008, eta: 0:13:25, time: 0.434, data_time: 0.005, memory: 2520, loss_rpn_cls: 0.0002, loss_rpn_bbox: 0.0010, loss_cls: 0.0110, acc: 99.5830, loss_bbox: 0.0093, loss: 0.0215
2019-10-18 04:29:04,350 - INFO - Epoch [12][2450/4269]	lr: 0.00008, eta: 0:13:04, time: 0.433, data_time: 0.005, memory: 2520, loss_rpn_cls: 0.0001, loss_rpn_bbox: 0.0009, loss_cls: 0.0088, acc: 99.6631, loss_bbox: 0.0082, loss: 0.0181
2019-10-18 04:29:26,013 - INFO - Epoch [12][2500/4269]	l

2019-10-18 04:40:36,791 - INFO - Epoch [12][4050/4269]	lr: 0.00008, eta: 0:01:34, time: 0.433, data_time: 0.005, memory: 2520, loss_rpn_cls: 0.0003, loss_rpn_bbox: 0.0010, loss_cls: 0.0105, acc: 99.6084, loss_bbox: 0.0087, loss: 0.0205
2019-10-18 04:40:58,467 - INFO - Epoch [12][4100/4269]	lr: 0.00008, eta: 0:01:12, time: 0.434, data_time: 0.005, memory: 2520, loss_rpn_cls: 0.0003, loss_rpn_bbox: 0.0010, loss_cls: 0.0104, acc: 99.6338, loss_bbox: 0.0084, loss: 0.0202
2019-10-18 04:41:20,115 - INFO - Epoch [12][4150/4269]	lr: 0.00008, eta: 0:00:51, time: 0.433, data_time: 0.005, memory: 2520, loss_rpn_cls: 0.0002, loss_rpn_bbox: 0.0010, loss_cls: 0.0086, acc: 99.6816, loss_bbox: 0.0079, loss: 0.0177
2019-10-18 04:41:41,742 - INFO - Epoch [12][4200/4269]	lr: 0.00008, eta: 0:00:29, time: 0.433, data_time: 0.005, memory: 2520, loss_rpn_cls: 0.0003, loss_rpn_bbox: 0.0009, loss_cls: 0.0100, acc: 99.5986, loss_bbox: 0.0090, loss: 0.0202
2019-10-18 04:42:03,408 - INFO - Epoch [12][4250/4269]	l